# Importing packages

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import collections as c
import os
import re
import numpy as np
import scipy.spatial.distance as sp
import math

ps = PorterStemmer() 
sno = SnowballStemmer("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Get list of file names

In [4]:
filenames = []
for fn in os.listdir("enron1\\ham"):
    if fn.endswith(".txt"):
        filenames.append(os.path.join("enron1\\ham", fn.replace("\\", "/")))
        
ham_num = len(filenames)

for fn in os.listdir("enron1\\spam"):
    if fn.endswith(".txt"):
        filenames.append(os.path.join("enron1\\spam", fn.replace("\\", "/")))
    
spam_num = len(filenames) - ham_num

In [5]:
len(filenames)

5172

# Stem email text

In [6]:
#flag to set initial df
first = True
stems_list = []
#iterate through each file
for fn in filenames:  
    #read file
    with open(fn, 'r', encoding="ISO-8859-1") as file:
        
        #remove numbers
        data = re.sub('[0-9]', '', file.read().replace('\n', ' '))
        
        #data = file.read().replace('\n', ' ')
        
        #tokenize + word-stem
        nltk_tokens = word_tokenize(data)
        stems = ""
        for w in nltk_tokens:
            stems += ps.stem(w) + " "
        
        stems_list.append(stems)


# Vectorize counts and get dictionary

In [7]:
#count vectorize
vect = CountVectorizer(input="content")
temp = vect.fit_transform(stems_list)
word_bag = vect.get_feature_names()



In [8]:
df = pd.DataFrame(temp.toarray(), columns=word_bag)
df

,aa,aaa,aabda,aabvmmq,aac,aachecar,aaer,aafco,aaiab,aaigrcrb,...,zynv,zyqtaqlt,zyrtec,zyyqywp,zzezrjok,zzn,zzo,zzocb,zzso,zzsyt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#dropping stop words

if drop_stop:
    nonstop_attr = df.columns.copy()
    for word in stopwords.words('english'):
        if word in df.columns:
            nonstop_attr = nonstop_attr.drop(word)
else:
    nonstop_attr = df.columns

In [10]:
len(df.columns)-len(nonstop_attr)

119

# Divide data into ham, spam, test, and train

In [498]:
df_nonstop = df[nonstop_attr].copy()[nonstop_attr]

ham_data = df_nonstop[:ham_num]
spam_data = df_nonstop[ham_num:]
ham_data["SPAM_LABEL"] = 0
spam_data["SPAM_LABEL"] = 1

msk = np.random.rand(len(ham_data)) < 0.7
ham_train = ham_data[msk]
ham_test = ham_data[~msk]

msk = np.random.rand(len(spam_data)) < 0.7
spam_train = spam_data[msk]
spam_test = spam_data[~msk]

train_data = pd.concat([ham_train, spam_train])
test_data = pd.concat([ham_test, spam_test])

C:\Users\janep\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\janep\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# K Nearest Neighbors

Setting up parameters

In [492]:
#setting parameters
k = 5
attr = train_data.columns.drop("SPAM_LABEL")

#copying data
train_data_knn = train_data.copy()
test_data_knn = test_data.copy()

#get norms
norms = sp.cdist(test_data_knn, train_data_knn)

Getting norms...
Getting neighbors...


In [496]:
def KNN(k, data):
    y = []
    neighbors = np.apply_along_axis(np.argpartition, 1, norms, k)[:,:k]
    for i in range(len(test_data_knn.index)):
        y.append(train_data_knn.iloc[list(neighbors[i]), :]["SPAM_LABEL"].mode()[0])
    return y

In [497]:
y = KNN(5, test_data_knn)
(y==test_data_d["SPAM_LABEL"]).sum()/test_data_d.shape[0]

0.8883216336949585

# Naive Bayes

We use add-one Laplace smoothing and log the probabilities to avoid zero-frequency problem.

In [17]:
def bayes_prob(row, freq, attr, total):
    p = 0
    for i in range(len(attr)):
        #using add-one laplacian smoothing
        p += math.log((freq[i][row[i+1]]+1)/total)
        
    return p


In [18]:
#get training data
train_data_b0 = train_data[train_data["SPAM_LABEL"]==0]
train_data_b1 = train_data[train_data["SPAM_LABEL"]==1]

#build frequency table
freq_0 = [c.Counter(train_data_b0[col]) for col in attr]
freq_1 = [c.Counter(train_data_b1[col]) for col in attr]

#get counts
counts = train_data["SPAM_LABEL"].value_counts()
total = counts.sum()

#get test data
test_data_bayes = test_data.copy()
test_data_bayes["SPAM_LABEL_y"] = -1


In [19]:
#classiying
for row in test_data.itertuples():
    p_0 = bayes_prob(row, freq_0, attr, counts[0]) + math.log(counts[0]/total)
    p_1 = bayes_prob(row, freq_1, attr, counts[1]) + math.log(counts[1]/total)
    
#     print(row[0])
#     print(p_0)
#     print(p_1)
    
    #give label
    if p_0 > p_1:
        test_data_bayes["SPAM_LABEL_y"].loc[row[0]] = 0
    else:
        test_data_bayes["SPAM_LABEL_y"].loc[row[0]] = 1

In [20]:
#accuracy
len(test_data_bayes[test_data_bayes["SPAM_LABEL_y"] == test_data_bayes["SPAM_LABEL"]].index)/len(test_data_bayes.index)

0.9115384615384615

# Decision tree

Lots of features = lots of time. We trim space by looking at topmost features with highest variance, then consider limited number of thresholds within each feature rather than every possible threshold. 

In [208]:
#copy data
train_data_temp = train_data.copy()
test_data_d = test_data.copy()

#split train and validation sets
msk = np.random.rand(len(train_data_temp)) < 0.9
train_data_d = train_data_temp[msk]
val = train_data_temp[~msk]



In [283]:
#get tree attributes
feat_num = round(len(train_data_d.columns)*.05)
tree_attr = list(train_data_d.var().sort_values(ascending=False)[:feat_num].index)
tree_attr.remove("SPAM_LABEL")

In [485]:
'''
Tree Class
'''
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.feature = None
        self.threshold = None
        self.label = -1

'''
Calculates Gini impurity
'''        
def gini(p):
    return 1-(p**2+(1-p)**2)


'''
Uses ID3 algorithm to build tree
'''
def ID3(data, y, attr, n):
    t = Tree()
    l = data.shape[0]
    
    #if all data is ham
    if data[y].sum() == 0:
        t.label = 0
        return t
    
    
    #if all data is spam
    elif data[y].sum() == l:
        t.label = 1
        return t
    
    #if we're out of attributes to train on
    elif len(attr) == 0 or n == 0:
        t.label = int(round(data[y].sum()/l))
        return t

    else:
        #get best feature and threshold
        best_f = None
        best_t = None
        best_g = -1
        
        #calculate gini ipurity of whole cell
        p = data[y].sum()/l
        g_total = gini(p)
        
        for a in attr:
            #get data and thresholds to try
            temp = np.array(list(zip(data[a], data[y])))
            thr_list = np.unique(data[a].values)
            
            for thr in thr_list:
                #split cells
                left = temp[temp[:,0] <= thr]
                right = temp[temp[:,0] > thr]
                
                #calculate gini impurity of each
                p_l = np.sum(left[:,1])/l
                p_r = np.sum(right[:,1])/l
                g =  g_total-(left.shape[0]/l*gini(p_l)+right.shape[0]/l*gini(p_r))
                
                #if it's the best feature:
                if g > best_g:
                    best_f = a
                    best_t = thr
                    best_g = g
        
        #set tree node parameters
        print("Best feature: " + a)
        t.feature = best_f
        t.threshold = best_t
        
        #split data into cells
        left_data = data[data[best_f] <= best_t]
        right_data = data[data[best_f] > best_t]
        
        #if one cell is empty
        if left_data.shape[0] == 0 or right_data.shape[0] == 0:
            t.label = int(round(data[y].sum()/l))
            return t
        
        #build subtree
        else:
            new_attr = attr.copy()
            new_attr.remove(best_f)
            t.left = ID3(left_data, y, new_attr, n-1)
            t.right = ID3(right_data, y, new_attr, n-1)
        
    return t
        
'''
Uses tree to classify data
'''
def tree_classify(t, test_data):
    y = []
    root = t
    for i in range(test_data.shape[0]):
        t = root
        x = test_data.iloc[i]
        while t.label == -1:
            if x[t.feature] <= t.threshold:
                t = t.left
            else:
                t = t.right
        y.append(t.label)
    return y

'''
Print tree (preorder)
'''

def show(t):
    if t.label == -1:
        print(t.feature, ": ", t.threshold)
        show(t.left)
        show(t.right)
    else:
        print("Label: " + t.label)

In [486]:
t = ID3(train_data_d, "SPAM_LABEL", tree_attr, 100)

Getting feature
ect
0
0.07320927530996829
ga
0
0.0921472839544834
ha
0
0.09519844937642236
final: student
Splitting cells
2635
615
Getting feature
ect
0
0.0595263370297161
ga
0
0.07777871297046057
hpl
0
0.09338787883562222
final: student
Splitting cells
2061
574
Getting feature
ect
0
0.06629329278828672
ga
0
0.07288108500625229
pm
0
0.07729258152512863
daren
0
0.07997239396682798
final: student
Splitting cells
1717
344
Getting feature
ect
0
0.03629180423013062
ga
0
0.0568320773666009
www
0
0.05866738688235529
price
0
0.06143859343041408
time
0
0.06359348926097896
attach
0
0.06419495249767548
final: student
Splitting cells
1493
224
Getting feature
ect
0
0.026411256067041755
meter
0
0.030912828885682808
ga
0
0.04868832121561606
final: student
Splitting cells
1343
150
Getting feature
ect
0
0.018940313133480546
meter
0
0.023228685918419534
pm
0
0.03166091433010981
chang
0
0.03388900112677051
let
0
0.0347493501660176
final: student
Splitting cells
1216
127
Getting feature
ect
0
0.0136453751

texaco
0
0.00013998179197656502
final: student
Splitting cells
802
1
Getting feature
hou
0
0.0
stacey
0
0.00013757885806127412
final: student
Splitting cells
801
1
Getting feature
hou
0
0.0
devon
0
0.0001351565914714803
final: student
Splitting cells
800
1
Getting feature
hou
0
0.0
man
3
0.00013271484374999254
final: student
Splitting cells
799
1
Getting feature
hou
0
0.0
dfarmer
0
0.000130253465185734
final: student
Splitting cells
798
1
Getting feature
hou
0
0.0
ua
1
0.00012777230480173762
final: student
Splitting cells
797
1
Getting feature
hou
0
0.0
vacat
1
0.00012527121034326683
final: student
Splitting cells
796
1
Getting feature
hou
0
0.0
realloc
0
0.00012275002826561687
final: student
Splitting cells
795
1
Getting feature
hou
0
0.0
readi
1
0.00012020860372183306
final: student
Splitting cells
794
1
Getting feature
hou
0
0.0
glover
0
0.00011764678055040112
final: student
Splitting cells
793
1
Getting feature
hou
0
0.0
bammel
0
0.00011506440126264617
final: student
Splitting cell

final: student
Splitting cells
205
19
Getting feature
ect
0
0.008025884708579376
ga
0
0.008772478634958853
corp
0
0.012692096748451107
price
0
0.018941106484235526
final: student
Splitting cells
187
18
Getting feature
ect
0
0.0036212364106145804
ga
0
0.004300218237604816
corp
0
0.006563490994238929
oil
0
0.010523606623008996
special
0
0.010523606623008998
final: student
Splitting cells
185
2
Getting feature
ect
0
0.001859870096539198
ga
0
0.0021504747991234474
corp
0
0.0033710145499772958
oil
0
0.010287406471482437
plan
0
0.010578011174066686
benefit
0
0.010694253055100387
final: student
Splitting cells
184
1
93
Getting feature
ect
0
0.0
ga
0
0.25
final: student
Splitting cells
1
1
93
94
Getting feature
ect
0
0.12071330589849108
low
0
0.1755829903978052
final: student
Splitting cells
16
2
94
95
Getting feature
ect
0
0.0408222772998979
td
0
0.06764834523983082
size
0
0.07989502842980017
compani
0
0.16678816153958284
mail
0
0.17495261699956244
ticket
0
0.18719930018953185
th
0
0.20236186

3
1
92
93
94
Getting feature
ect
0
0.09375
final: student
Splitting cells
6
2
94
95
Getting feature
ect
0
0.05555555555555558
ga
0
0.08333333333333331
compani
0
0.11728395061728397
corp
0
0.1388888888888889
one
0
0.1481481481481482
account
0
0.1635802469135803
contact
0
0.16666666666666674
regard
0
0.19444444444444448
final: student
Splitting cells
11
7
Getting feature
ect
0
0.05409466566491361
ga
0
0.08114199849737042
compani
0
0.18933132982719766
account
1
0.2434259954921113
final: student
Splitting cells
9
2
94
95
96
97
98
Getting feature
ect
0
0.10465425605921475
ga
0
0.1287426329409801
compani
0
0.1792381111389375
compani
1
0.18228967359545856
invest
0
0.18401001753067858
final: student
Splitting cells
114
51
Getting feature
ect
0
0.10810343803491493
ga
0
0.11428077735119582
product
0
0.13646304125965897
call
0
0.14733548244264105
time
0
0.15162560139962106
final: student
Splitting cells
63
51
Getting feature
ect
0
0.13216715257531586
daren
0
0.15664255120037437
final: student
Spl

In [487]:

        print(t.label)
z = tree_classify(t, test_data_d)

In [478]:
test_data_d["SPAM_LABEL"].sum()

449

In [491]:
#accuracy
(z==test_data_d["SPAM_LABEL"]).sum()/test_data_d.shape[0]


0.9132099553286535